In [1]:
import numpy as np
import pandas as pd
import re
import jieba.posseg as pseg
import  math
#simtype = 'COS'
simtype = 'COS'
#simtype = 'BERT'

#my_regex = "\（.*?\）|\(([^\)0-9零一二三四五六七八九十]?|[^\).]{2,})\)"
my_regex = r"○{2,}"

#tmpstr = '丙○○意圖為自己不法之所有，(甲)YY(假)XX(十一)(一)a(一一)b(一一一)c(年一)d(一年一)e(一一一年)A(9年)B(99年)C(999年)A(AAA)於民國105年10月8日晚間7時20分許，騎乘登記於不知情之王介平名下之車牌號碼000-000號普通重型機車，行經高雄市○○區○○○路000號前時，將口罩脫下遮掩車牌，自後靠近步行在人行道上之甲○○，趁甲○○不及防備之際，以左手徒手掠取甲○○右手持有之背包1只（內有現金新臺幣【下同】9500元、索尼牌ZR綠色及Xperformance金色行動電話共2具【IMEI：000000000000000、000000000000000號】、長夾1個、信用卡及金融卡共5張、國民身分證2張、國民健康保險卡1張、普通重型機車駕照1張、識別證2張等物）得手後逃逸。嗣因丙○○將背包內之現金取走後，將背包棄置在高雄市鳳山區武營路某處，再經民眾在高雄市鳳山區衛武營附近拾獲上開背包（內有長夾1個、國民身分證2張、國民健康保險卡1張、信用卡3張、金融卡1張等物），遂送交警察機關發還甲○○領回。(二)於同年10月10日晚間11時5分許，騎乘向不知情之劉伊恩借用之車牌號碼000-0000號普通重型機車，行經高雄市○○區○○○路000號前時，使用口罩遮掩車牌，自後靠近步行在人行道上之少年乙○○（89年生，基於兒童及少年福利與權益保障法第69條第2項規定不得揭露足以識別少年身分之資訊，年籍詳卷），趁乙○○不及防備之際，以左手徒手掠取乙○○右手持有之背包1只（內有皮夾1只、現金3萬2000元、提款卡1張、國民身分證1張、國民健康保險卡1張等物）得手後逃逸，並將現金取出，其餘物品則丟棄。嗣為警循線於105年10月16日凌晨2時許，在高雄市○○區○○路000巷00號前查獲，並扣得現金2600元。'
#print((tmpstr))
#print(len(tmpstr))
#print((re.sub(my_regex, "", tmpstr)))
#print(len(re.sub(my_regex, "", tmpstr)))
p_upper = 2.5
p_upperT = "25"
filedir = 'data_tfidf3'
filenum = '5K'
filename = 'Result_' + filenum
print(filenum , '-', p_upper)

berttype = 'bert-base-chinese'
#berttype = 'voidful/albert_chinese_base'
#berttype = 'wptoux/albert-chinese-large-qa'
#berttype = 'clue/roberta_chinese_base'


testtype = 0

tmpSpecial = ''
max = 20
top_k = 15

5K - 2.5


In [2]:
dt_idf = pd.read_csv("..//TFIDF//" + filedir +"//Result_TFIDF_" + filenum +".csv", encoding='utf8')
#dt_idf = pd.read_csv("..//TFIDF//data_tfidf2//Result_TFIDF_100000X.csv", encoding='utf8')
dt_idf = dt_idf[dt_idf["P"] > p_upper].drop(columns=['詞性']).set_index('詞').T.to_dict('list')


In [3]:
#df=pd.read_csv('./data_tfidf/result_4000_3.csv', index_col=0)
df=pd.read_csv("..//TFIDF//" + filedir +"/result_Sbert_" + filenum +"_TFIDF_" + p_upperT +"X.csv")
df = df.dropna()
#df=pd.read_csv('.//data//Result_R5.csv', index_col=0)
df.head(2)

,court,date,no,sys,reason,type,mainText,judgement,Situation
0,臺灣桃園地方法院,2018-04-18T00:00:00+08:00,"106,易,1093",刑事,妨害公務,判決,邱建豪犯妨害公務執行罪，累犯，處拘役拾日，如易科罰金，以新臺幣壹仟元折算壹日。,臺灣桃園地方法院刑事判決106年度易字第1093號公訴人臺灣桃園地方法院檢察署檢察官被告邱建...,邱建豪於民國106年5月1日下午5時15分許，在桃園市○○區○○路000號桃園市政府警察局桃...
1,臺灣臺北地方法院,2018-12-26T00:00:00+08:00,"107,侵簡,3",刑事,妨害性自主,判決,余暐勝對於十四歲以上未滿十六歲之女子為性交，共肆罪，各處有期徒刑參月。應執行有期徒刑捌月。緩...,臺灣臺北地方法院刑事簡易判決107年度侵簡字第3號公訴人臺灣臺北地方檢察署檢察官被告余暐勝選...,余暐勝於民國107年3月間透過網路交友軟體「BEETALK」結識代號000之14歲以上未滿1...


In [4]:
test= df.sample(n=200)
test.head(2)

,court,date,no,sys,reason,type,mainText,judgement,Situation
4261,臺灣桃園地方法院,2018-09-17T00:00:00+08:00,"106,易,665",刑事,毀損,判決,本件公訴不受理。,臺灣桃園地方法院刑事判決106年度易字第665號公訴人臺灣桃園地方檢察署檢察官被告張永和上列...,被告張永和於民國105年6月1日凌晨5時3分許，在桃園市○○區○○○路000巷0弄00號前，...
4780,臺灣新北地方法院,2017-12-15T00:00:00+08:00,"106,訴,362",刑事,槍砲彈藥刀械,判決,詹基源未經許可，持有可發射子彈具有殺傷力之改造手槍，累犯，處有期徒刑肆年陸月，併科罰金新臺幣...,臺灣新北地方法院刑事判決106年度訴字第362號公訴人臺灣新北地方法院檢察署檢察官被告詹基源...,詎猶不知悔改，明知可發射子彈具有殺傷力之改造手槍及具有殺傷力之子彈，分別為槍砲彈藥刀械管制條...


In [5]:
test.to_csv("..//TFIDF//" + filedir +"//Result_SBert_" + filenum +"_Test.csv")

In [6]:
test = pd.read_csv("..//TFIDF//" + filedir +"//Result_SBert_" + filenum +"_Test.csv", encoding='utf8')
test.head(2)

,Unnamed: 0,court,date,no,sys,reason,type,mainText,judgement,Situation
0,4261,臺灣桃園地方法院,2018-09-17T00:00:00+08:00,"106,易,665",刑事,毀損,判決,本件公訴不受理。,臺灣桃園地方法院刑事判決106年度易字第665號公訴人臺灣桃園地方檢察署檢察官被告張永和上列...,被告張永和於民國105年6月1日凌晨5時3分許，在桃園市○○區○○○路000巷0弄00號前，...
1,4780,臺灣新北地方法院,2017-12-15T00:00:00+08:00,"106,訴,362",刑事,槍砲彈藥刀械,判決,詹基源未經許可，持有可發射子彈具有殺傷力之改造手槍，累犯，處有期徒刑肆年陸月，併科罰金新臺幣...,臺灣新北地方法院刑事判決106年度訴字第362號公訴人臺灣新北地方法院檢察署檢察官被告詹基源...,詎猶不知悔改，明知可發射子彈具有殺傷力之改造手槍及具有殺傷力之子彈，分別為槍砲彈藥刀械管制條...


In [7]:
df1 = df[~(df['court'] + df['no']).isin(test['court'] + test['no'])]
df1['reason'] = df1['reason'].str.replace('等','')
df1['reason'] = df1['reason'].str.replace('罪','')
df1['reason'] = df1['reason'].str.replace("條例","")
df1['reason'] = df1['reason'].str.replace("防制","")
df1['reason'] = df1['reason'].str.replace("管制","")
df1['reason'] = df1['reason'].str.replace("違反","")
df1['Situation'] = df1['Situation'].str.replace(my_regex, "")

c:\users\user\pycharmprojects\pytorch\venv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\user\pycharmprojects\pytorch\venv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\user\pycharmprojects\pytorch\venv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [8]:
df1.head()

,court,date,no,sys,reason,type,mainText,judgement,Situation
0,臺灣桃園地方法院,2018-04-18T00:00:00+08:00,"106,易,1093",刑事,妨害公務,判決,邱建豪犯妨害公務執行罪，累犯，處拘役拾日，如易科罰金，以新臺幣壹仟元折算壹日。,臺灣桃園地方法院刑事判決106年度易字第1093號公訴人臺灣桃園地方法院檢察署檢察官被告邱建...,邱建豪於民國106年5月1日下午5時15分許，在桃園市區路000號桃園市政府警察局桃園分局埔...
1,臺灣臺北地方法院,2018-12-26T00:00:00+08:00,"107,侵簡,3",刑事,妨害性自主,判決,余暐勝對於十四歲以上未滿十六歲之女子為性交，共肆罪，各處有期徒刑參月。應執行有期徒刑捌月。緩...,臺灣臺北地方法院刑事簡易判決107年度侵簡字第3號公訴人臺灣臺北地方檢察署檢察官被告余暐勝選...,余暐勝於民國107年3月間透過網路交友軟體「BEETALK」結識代號000之14歲以上未滿1...
2,臺灣臺中地方法院,2011-05-19T00:00:00+08:00,"100,中交簡,1002",刑事,公共危險,判決,王炎權服用酒類，不能安全駕駛動力交通工具而駕駛，處拘役伍拾日，如易科罰金，以新臺幣壹仟元折算壹日。,臺灣臺中地方法院刑事簡易判決100年度中交簡字第1002號聲請人臺灣臺中地方法院檢察署檢察官...,王炎權於民國100年4月8日晚間9時30分許起至翌日凌晨3時許止，在彰化縣芬園鄉路○段291...
3,臺灣臺中地方法院,2011-01-25T00:00:00+08:00,"100,中交簡,100",刑事,公共危險,判決,林耀明服用酒類，不能安全駕駛動力交通工具而駕駛，處拘役伍拾伍日，如易科罰金，以新臺幣壹仟元折...,臺灣臺中地方法院刑事簡易判決100年度中交簡字第100號聲請人臺灣臺中地方法院檢察署檢察官被...,林耀明於民國99年10月22日下午4時許起至5時許止，在臺中市區路與豐樂路交岔路口附近之某工...
4,臺灣臺中地方法院,2011-05-12T00:00:00+08:00,"100,中交簡,1016",刑事,公共危險,判決,朱建鴻犯不能安全駕駛動力交通工具罪，處罰金新台幣伍萬元，如易服勞役，以新臺幣壹仟元折算壹日。,臺灣臺中地方法院刑事簡易判決100年度中交簡字第1016號聲請人臺灣臺中地方法院檢察署檢察官...,朱建鴻於民國100年4月19日晚間9時許起，在臺中市大雅區清泉崗友人住處內飲用啤酒，至同日晚...


In [13]:
df1.loc[875].j

'賭博'

In [40]:
print("總筆數",df1['reason'].count(), "\n")
df1.loc[:,'reason'].value_counts()

總筆數 5558 



公共危險      334
著作權法      326
毀損        324
竊盜        322
妨害公務      320
妨害名譽      314
毒品危害      313
妨害風化      298
妨害性自主     281
妨害自由      265
槍砲彈藥刀械    246
賭博        236
侵占        236
傷害        233
偽造文書      224
過失傷害      211
搶奪        210
殺人未遂      208
詐欺        181
商標法       175
強盜        154
殺人         41
毀棄損壞       35
恐嚇         31
恐嚇取財       19
貪污          7
過失致死        7
醫師法         3
藥事法         2
替代役實施       1
偽證          1
Name: reason, dtype: int64

In [41]:
df.count()

court        5760
date         5760
no           5760
sys          5760
reason       5760
type         5760
mainText     5760
judgement    5760
Situation    5760
dtype: int64

In [42]:
test.count()

Unnamed: 0    200
court         200
date          200
no            200
sys           200
reason        200
type          200
mainText      200
judgement     200
Situation     200
dtype: int64

In [43]:
df1.count()

court        5558
date         5558
no           5558
sys          5558
reason       5558
type         5558
mainText     5558
judgement    5558
Situation    5558
dtype: int64

In [44]:
#xxxx = df1.loc[:,'reason'].value_counts()
#for a,b in xxxx.items():
#    print(a, b)

In [45]:
import pandas as pd
import time
tStart = time.time()

In [46]:
class Data(object):
    def __init__(self):
        self.no = "0"
        self.reason = ""
        self.Situation = ""

In [47]:
#embedder  SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Corpus with example sentences
#corpus  ['A man is eating food.',
#          'A man is eating a piece of bread.',
#          'The girl is carrying a baby.',
#          'A man is riding a horse.',
#          'A woman is playing violin.',
#          'Two men pushed carts through the woods.',
#          'A man is riding a white horse on an enclosed ground.',
#          'A monkey is playing drums.',
#          'A cheetah is running behind its prey.'
#          ]
corpus = []
corpusList = []
for index, row in df1.iterrows():
    if (row.Situation == None):
        continue
    if (isinstance(row.Situation, float)):
        continue
    if (len(row.Situation) == 0):
        continue
    if (index == 5):
        print()
    d = Data()
    d.no = row.no
    d.reason = row.reason
    d.Situation = row.Situation
    corpusList.append(d)
    corpus.append(row.Situation)


print(len(corpus))


5558


In [48]:
tEnd = time.time()#計時結束
#列印結果
print("轉向量時間", "%f sec" % (tEnd - tStart))

轉向量時間 1.059202 sec


In [49]:
def doRule(Situation):

    my_regex = r"[\(\（]([^一二三四五六七八九零十1234567890]{1}[^\)\）]+|[一二三四五六七八九零十1234567890]{1}[^\)\）]{1,})[\)\）]"
    Situation = Situation.replace(my_regex, "")
    #/○{2,}/
    regex1 = r"/○{2,}"
    Situation = Situation.replace(regex1, "")

    #/\d{10}/
    regex2 = r"\d{10}"
    Situation = Situation.replace(regex2, "")

    #/[A-Z0-9]{2,3}\-[A-Z0-9]{3,4}/ 車牌
    regex3 = r"[A-Z0-9]{2,3}\-[A-Z0-9]{3,4}"
    Situation = Situation.replace(regex3, "")
    Source = Situation
    try:
        words =pseg.cut(Situation)
        seq = ''
        for w, l in words:
            if (w in dt_idf):
                continue
            elif (l in ('m', 't', 'c')):
                continue
            else:
                seq += w
        Situation = seq
    except:
        Situation = Source
    return Situation

In [50]:
#specialtest  ='8旬的余姓老婦，獨自坐在活動中心的椅子上休息，突然就出現了一名伊姓男子，他從側門進入後，先是到飲水機喝水，隨後拿起鋁製報夾瘋狂地朝余婦猛打，余婦一度想起身制止並離開，伊男卻不予理會並朝余婦後膝猛打，導致余婦當場腿軟跪下求饒，豈料，伊男仍不罷休持續朝余婦背部、頭部狂踹，導致現場鮮血直流，警方獲報後趕到現場，連用2次辣椒水，才將不斷反抗的伊男壓制在地，並將人帶回偵訊，訊後依殺人未遂、傷害罪嫌送辦。'
#specialtest  ='基隆張男未依規定服役遭函送基隆地檢署偵辦，他為證明已主動向轄區區公所申請入營服役，沒有躲兵役的意圖，在民國107年11月間在住處透過網路取得「基隆市研發替代役入營通知書」電子檔，以修圖軟體變造個資內容，偽造出基隆巿政府核發給他的研發替代役入營通知書，再彩色列印出來'
#specialtest  ='吳姓婦人與胡姓先生結婚後育有2子，吳婦的兒子從15樓租屋處，把東西丟下樓，還好沒砸到人，吳婦獲知後，一時惱怒持衣架打兒子屁股，體罰兒子打到瘀青。'
#specialtest  ='曾姓男子昨天上午10點多，騎機車在路上尋找年輕女子，鎖定3名路過女大生後，藉由問路露出自己的生殖器，女大生嚇傻通報學校，學校報警後，警方調閱監視器畫面，掌握嫌犯所騎的機車和男子特徵，到曾男住處逮人，曾男坦承犯行，最後被依妨害風化罪函送。曾男到案說明後坦承犯案，他表示，自己打零工維生，當天剛好沒上班，出門閒晃，一時興起就做了這個舉動，他坦言自己非常後悔。'
#specialtest  ='曾姓男子因與翁姓女子有感情糾紛而心生不滿，在去年7月19日晚間9時許，騎機車到台中市豐原區鎌村路附近，並以機車擋住翁女去路，雙方談判後，曾趁翁女不注意，搶走翁女的iPhone XS手機，並先將手機丟在案發地點附近草叢內，曾犯案後離開現場。'

# Query sentences:
#queries  ['A man is eating pasta.', 'Someone in a gorilla costume is playing a set of drums.', 'A cheetah chases prey on across a field.']
queries = []
queries_ori = []
#specialtest  ='被告陳守仁為高職畢業擔任保全之男子，竟不思以正常手段獲取財物，因一時貪念，以徒手竊取該超商內商品供己食用之犯罪動機、目的、手段、生活狀況、智識程度、所竊得物品價值新臺幣（下同）總共約821元'
specialtest  ='台南永康區陸軍砲兵訓練指揮部蔡姓志願役士兵，前年底涉利用前女友帳號，將他在營區內拍攝軍用機識別標識、軍械室監視器畫面等機密資料，傳送至陸軍司令部官方LINE帳號，留言要求砲訓部人員聯繫，恫嚇對方交付30萬元，否則將向媒體爆料其軍紀不佳，砲訓部向警方報案，查出蔡因涉酒後駕車公共危險案，遭汰除退伍而挾怨報復，檢方本月初依恐嚇取財未遂罪嫌起訴。'
queries_ori.append(specialtest)
specialtest = doRule(specialtest)
queries.append(specialtest)
specialtest  ='台南市官田區南58線今天（17日）上午7點46分發生要命的車禍，1名女機車騎士疑撞上倒車的大貨車後車斗，人車倒地，頭部重創，送醫後不治，確切車禍原因由麻豆警分局調查釐清。遇到大貨車倒車，一定要特別當心！台南就有一名年僅30歲的女騎士，趕著上班途中，未留意到一旁的產業道路有大貨車正在倒車，迎頭撞了上去，結果頭部重創，緊急送醫搶救仍宣告不治。'
queries_ori.append(specialtest)
specialtest = doRule(specialtest)
queries.append(specialtest)
specialtest  ='南投縣埔里鎮一名60歲李姓男子13日跟友人前往KTV唱歌，被女童家長懷疑猥褻5歲的女兒，李姓男子被帶到八德夜市附近毆打，重傷送醫，延到16日死亡，檢警展開偵辦，經訊問相關犯嫌後，女童父母各以3萬元交保候傳。南投地檢署表示，受理通報案件是有女童疑似遭強制猥褻，經女童家屬發現認為是李姓男子所為，於是予以毆打。而李姓男子在16日死亡，檢察官已經進行相驗，進一步調查。據了解，李姓男子是跟友人前往KTV，與女童的父母一起唱歌。但是，李姓男子跟女童父母之前並不認識，女童父母曾到外面抽菸，此時，李姓男子被拍到牽著女童從後門出去，女童回到包廂告知父母被猥褻。不過，李姓男子家屬表示，監視器只拍到牽著女童的手，懷疑李姓男子是被冤枉。由於李姓男子已經死亡，真相有待檢警釐清。'
queries_ori.append(specialtest)
specialtest = doRule(specialtest)
queries.append(specialtest)
specialtest  = '桃園一名王姓男子2018年7月覬覦死黨22歲女友小青（化名），以幫忙找工作為由，邀對方到家中，先後性侵2次得逞。期間小青一度向男友求救，不料王男竟搶走手機，偽裝玩笑傳訊「整到你了」。隔天，王男妻子要求見面談和解，恫嚇要告「通姦罪」，還硬塞3000元「強行和解」，不料此舉反成判刑鐵證。當時小青用電腦上網找工作，王男見有機可趁，忍不住將頭靠過去，伸手撫摸女方的腰，並順勢游移到大腿內側。小青驚覺有異，出聲制止「你幹嘛碰我」，怎料下秒就被推到床上性侵。小青難過地說，當時王男強脫她的衣物，聽到她頻頻哭喊「不要」，僅回了一句「一次就好」，隨後性侵得逞。完事後，王男問她要不要出去吃飯，隨後在家煮麵，繼續和她討論找工作的事，讓小青很詫異，不禁心想「剛剛發生那種事，他還可以那麼冷靜」，但又害怕遭遇不測，不敢貿然離開。事後，小青趁隙傳訊給男友「老公好恐怖啦」、「ＯＯ是變態」、「他剛剛對我做了那種事！」當時男友在上班，第一時間傳訊詢問「所以是真的嗎？」不料王男見狀，竟恐嚇小青「妳知道這件事傳出去後果有多嚴重嗎？」隨後奪去手機，偽裝成小青回覆「有又怎樣，整到你很爽，你這個白癡！」還親自打電話說「只是開玩笑」。事發至此，小青很難過、害怕，要求立即離開。怎料王男意猶未盡，竟不顧小青哭泣、反抗，又將人壓在床上二次性侵。完事後，王男開車將人送回家，小青則向男友哭訴遭性侵，討論要不要報案。不過，王男妻子搶先一步，約許男及小青隔天見面談和解。'
queries_ori.append(specialtest)
specialtest = doRule(specialtest)
queries.append(specialtest)
specialtest  ='59歲張姓男子經常到新北市新莊區中港路的友人住處打麻將，16日晚間8時許，張男再度赴約打牌喝酒，過程中牌友55歲許姓男子不斷輸錢，火氣上來怒譙「X你老母」，張男也出言反嗆，雙方邊打牌邊互罵，場面幾乎失控，最後翻臉乾脆不打了。晚間11時，牌局不歡而散後，許男下樓打電話找來57歲吳姓男子，2人動手圍毆剛走出門的張男；張男雙手不敵四拳，跑回牌友家中廚房，隨手拿一把水果刀跑下樓，見許、吳2人要騎機車離開，立刻衝上前刺殺吳男右大腿，再捅許男右胸，逞兇後棄刀準備逃逸時，遭趕到的警員逮捕。警方逮捕張男後，立刻將受傷的2人協助到院治療，經緊急救治都沒有生命危險，張男也在衝突過程中，被毆傷鼻樑，造成5公分撕裂傷。由於兇殺現場剛好是麥當勞速食店門口，警方也找了目擊民眾詢問了解事發經過，並待3人治療包紮後帶回派出所，訊後將張男依殺人未遂罪移送偵辦。'
queries_ori.append(specialtest)
specialtest = doRule(specialtest)
queries.append(specialtest)
specialtest  ='屏東一名70多歲婦人，懷疑老公出軌，去年拿木頭砧板毆打丈夫致死，她沒叫救護車就離開案發現場，幾小時才向警方自首，婦人被依殺人罪送辦。木質圓形砧板，寬31，厚度4公分，隔熱用的木塊，卻成了殺人兇器，屏東一名70多歲婦人，跟71歲丈夫分居，懷疑老公出軌，去年十月，她跑到男子租屋處想抓姦，雙方爭吵，男子隨手拿木塊追進廚房，砧板掉落被婦人撿起，攻擊對方頭胸跟腹部，眼睜睜看著丈夫，頭破血流倒地，她沒叫救護車就離開，再通知女兒到場，但老翁已經傷重不治。'
queries_ori.append(specialtest)
specialtest = doRule(specialtest)
queries.append(specialtest)
specialtest  ='新北市中和一名男子遺失皮夾，調閱監視器發現被路過婦人撿走，警方十天後找到人，依侵占遺失物法辦。但男子的妻子不滿，這段時間每天抱著十月大的小孩，挨家挨戶調閱監視器，夫妻間也因此鬧不愉快，打算向嫌犯提告精神求償。紅衣婦人騎單車突然停下來，低頭一看有個黑色皮夾，她撿起來騎車離開，沒往派出所方向疑似直奔回家，皮夾主人心急如焚，因為裡面裝著一萬多元都是小孩的奶粉錢。丈夫的皮夾不見，施小姐每天抱著十月大的嬰兒，挨家挨戶調閱監視器，尋找婦人身影和警方分頭進行希望加快偵辦進度，名牌皮夾和裡頭的錢損失約三萬，十天後警方終於找到，撿走皮夾的徐姓婦人她辯稱忘了拿去派出所，被依侵占遺失物法辦，皮夾雖然物歸原主，但這段時間施小姐一家人四處奔走生活被打亂，這些看不見的損失，打算向對方求償精神撫慰金。'
queries_ori.append(specialtest)
specialtest = doRule(specialtest)
queries.append(specialtest)
specialtest  ='陳姓男子透露，死者曾在5天內輕生3次，都是他帶著就醫。4年多前，旅館發生槍擊案，警消到場，發現劉姓死者當場沒命，拉封鎖線，仔細採證，事後，認定死者的老闆陳姓男子涉嫌重大，檢警認為，犯案後去藏毒品，到另一家汽車旅館梳洗的陳姓男子，替死者餵食大量毒品，最後開槍殺人，動機跟感情因素有關，不過，被告從頭到尾都否認，強調槍、毒，都是死者的，只是一二審法官不認同這樣的說法，判刑十五年，案件上訴更一更二審，刑度少了一年，更三審法官認為，死者有服用毒品情況，現場沒綑綁打鬥痕跡，救護車還是被告叫的，傳喚證人後不排除，死者吸毒後還有意識，再加上案發前，死者與人微信語音聯絡，背景音聽得到被告睡覺在打呼。'
queries_ori.append(specialtest)
specialtest = doRule(specialtest)
queries.append(specialtest)
specialtest  ='今年4月7日晚間近7時許，陳姓遊覽車司機在台中市一間廟宇旁，見到6歲的女童身邊無大人帶著，於是尾隨至暗巷，強行抱走幼女，伸出狼爪指侵；事後孩子驚慌向家長哭訴，警方獲報，調閱路口監視器，鎖定陳嫌，70分鐘後順利將他逮捕到案。陳嫌落網後，坦承犯行，供稱：「非常後悔當時的衝動。」檢察官依「加重強制性交罪」將他起訴，且因陳男所涉是本刑5年以上重罪，有逃亡之虞，向法官聲請羈押獲准。明知女童年幼，竟為逞性欲，強抱、指侵，使被害人心裡產生負面影響，犯罪情節嚴重，危害治安甚鉅，加上他未與被害人達成民事和解，取得被害人家屬諒解。'
queries_ori.append(specialtest)
specialtest = doRule(specialtest)
queries.append(specialtest)
specialtest  ='男等3人受盧姓男子（另案苗栗地檢署偵辦中）、綽號「老闆」之人指示，以5萬元報酬，由温負責與盧協調報關事宜，再由温提供行動電話聯絡、楊提供寄送地址，劉則到場把風，今年7月2日到倉儲收取泰國寄來的毒品包裹，警方當場查獲，夾藏7包重1937.11公克、純度89.11%海洛因。3人主觀認為郵包最糟應該是愷他命，且今年都沒有出境紀錄，兼衡與綽號「老闆」通話紀錄，足見3人從未參與包裝包裹，無從認定主觀上有認識郵包內是海洛因，且3 人預期可獲得報酬僅為5萬元。'
queries_ori.append(specialtest)
specialtest = doRule(specialtest)
queries.append(specialtest)

for index, row in test.iterrows():
    queries.append(str(row.Situation))
    queries_ori.append(str(row.Situation))
print(len(queries))

210


In [51]:
print(queries_ori[20])

康昭仁於「世旻設計有限公司」擔任業務經理一職，於民國99年8月間某日起，於網路YES123求職網站上張貼招募人才之訊息。適有代號0女子，於同一網站上張貼求職之訊息，康昭仁遂撥打電話予A女，約定於99年9月6日見面，見面後康昭仁以要瞭解A女之文康活動能力為由，邀約A女於同日下午16時許前往錢櫃企業股份有限公司所經營，位於臺南市○○區○○路99號之KTV唱歌。嗣於同日下午17時許，在錢櫃KTV包廂內，康昭仁為滿足自己之性慾，基於強制猥褻之犯意，利用A女唱歌不注意之際，先以雙手自A女後方環抱A女之身體與雙手，繼而撫摸A女之腰部及背部，A女受驚嚇後不斷掙扎，但無法掙脫，因此向康昭仁騙稱其要點歌，康昭仁因而放手，待A女欲找尋包廂內之服務鈴通知服務人員到場時，康昭仁突又以右手手肘勒住A女脖子，右手手掌按住A女肩膀，繼而以左手伸進A女胸罩內撫摸A女之胸部，以此等強暴之方式使A女無法抗拒，違反A女意願而為猥褻之行為得逞，嗣因A女之手機鈴聲作響，康昭仁方始罷手。


In [ ]:
from sentence_transformers import SentenceTransformer, util
embedder=SentenceTransformer(berttype)
#embedder=SentenceTransformer('model_bert_4000X')

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [ ]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
i = -1
tmpSpecial = ''
show = 0

simtype = 'MD'

for query in queries:
    i += 1
    if (query == 'nan' or queries_ori[i] == 'nan'):
        print('break1')
        continue
    if (len(query) < 10 or len(queries_ori[i]) < 10):
        print('break2')
        continue


    tStart2 = time.time()
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    if (simtype == 'COS'):
        cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
        cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    elif (simtype == 'DIS'):
        cos_scores = util.pytorch_Dis(query_embedding, corpus_embeddings)
        cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(cos_scores, range(top_k))[0:top_k]
    elif (simtype == 'DPS'):
        cos_scores = util.pytorch_DPS(query_embedding, corpus_embeddings)
        cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    elif (simtype == 'BS'):
        cos_scores = util.pytorch_BS(query_embedding, corpus_embeddings)
        cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    elif (simtype == 'MD'):
        cos_scores = util.pytorch_Manhattan_Dis(query_embedding, corpus_embeddings)
        cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    elif (simtype == 'JS'):
        cos_scores = util.pytorch_Jaccard_Sim(query_embedding, corpus_embeddings)
        cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    else:
        break

    print("\n\n\n\n")
    for qlen in range(0, len(queries_ori[i]), 50):
        print(show + 1 if qlen==0 else '', "Query:" if qlen==0 else '\t', queries_ori[i][qlen : qlen + 50])
    #print("\nTop 5 most similar sentences in corpus:")
    print('')
    no = 1
    for idx in top_results[0:top_k]:
        if (corpus[idx] == specialtest):
            continue
        if (queries_ori[i] == corpusList[idx].Situation):
            continue
        if (isinstance(corpusList[idx].Situation, float)):
            continue
        if (corpusList[idx].Situation.strip() == 'nan'):
            continue

        print(no, "  "
              , corpusList[idx].no,"\t"
              ,  corpusList[idx].reason
              , "\t(Score: %.4f)" % (cos_scores[idx]))
        for qlen in range(0, len(corpusList[idx].Situation.strip()), 50):
            print(corpusList[idx].Situation.strip()[qlen : qlen + 50])
        #print (corpusList[idx].Situation.strip())
        #    #if (len(corpus[idx].strip()[0:100]) > 100):
        #    print (corpus[idx].strip()[0:100])
        #else:
        #    print (corpus[idx].strip()[0:100])
        #if (tmpSpecial == ''):
        #    tmpSpecial = corpus[idx].strip()
        #    print(tmpSpecial)
        print('\n')
        no = no + 1
        if (no > 5):
            break

    show += 1
    tEnd2 = time.time()#計時結束
    print("COS相似計算時間", "%f sec" % (tEnd2 - tStart2))

    if (show >= max):break

In [ ]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
i = -1
tmpSpecial = ''
show = 0

simtype = 'JS'

for query in queries:
    i += 1
    if (query == 'nan' or queries_ori[i] == 'nan'):
        print('break1')
        continue
    if (len(query) < 10 or len(queries_ori[i]) < 10):
        print('break2')
        continue


    tStart2 = time.time()
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    if (simtype == 'COS'):
        cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
        cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    elif (simtype == 'DIS'):
        cos_scores = util.pytorch_Dis(query_embedding, corpus_embeddings)
        cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(cos_scores, range(top_k))[0:top_k]
    elif (simtype == 'DPS'):
        cos_scores = util.pytorch_DPS(query_embedding, corpus_embeddings)
        cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    elif (simtype == 'BS'):
        cos_scores = util.pytorch_BS(query_embedding, corpus_embeddings)
        cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    elif (simtype == 'MD'):
        cos_scores = util.pytorch_Manhattan_Dis(query_embedding, corpus_embeddings)
        cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    elif (simtype == 'JS'):
        cos_scores = util.pytorch_Jaccard_Sim(query_embedding, corpus_embeddings)
        #cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    else:
        break

    print("\n\n\n\n")
    for qlen in range(0, len(queries_ori[i]), 50):
        print(show + 1 if qlen==0 else '', "Query:" if qlen==0 else '\t', queries_ori[i][qlen : qlen + 50])
    #print("\nTop 5 most similar sentences in corpus:")
    print('')
    no = 1
    for idx in top_results[0:top_k]:
        if (corpus[idx] == specialtest):
            continue
        if (queries_ori[i] == corpusList[idx].Situation):
            continue
        if (isinstance(corpusList[idx].Situation, float)):
            continue
        if (corpusList[idx].Situation.strip() == 'nan'):
            continue

        print(no, "  "
              , corpusList[idx].no,"\t"
              ,  corpusList[idx].reason
              , "\t(Score: %.4f)" % (cos_scores[idx]))
        for qlen in range(0, len(corpusList[idx].Situation.strip()), 50):
            print(corpusList[idx].Situation.strip()[qlen : qlen + 50])
        #print (corpusList[idx].Situation.strip())
        #    #if (len(corpus[idx].strip()[0:100]) > 100):
        #    print (corpus[idx].strip()[0:100])
        #else:
        #    print (corpus[idx].strip()[0:100])
        #if (tmpSpecial == ''):
        #    tmpSpecial = corpus[idx].strip()
        #    print(tmpSpecial)
        print('\n')
        no = no + 1
        if (no > 5):
            break

    show += 1
    tEnd2 = time.time()#計時結束
    print("COS相似計算時間", "%f sec" % (tEnd2 - tStart2))

    if (show >= max):break

In [ ]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
i = -1
tmpSpecial = ''
show = 0

simtype = 'DPS'

for query in queries:
    i += 1
    if (query == 'nan' or queries_ori[i] == 'nan'):
        print('break1')
        continue
    if (len(query) < 10 or len(queries_ori[i]) < 10):
        print('break2')
        continue


    tStart2 = time.time()
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    if (simtype == 'COS'):
        cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
        cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    elif (simtype == 'DIS'):
        cos_scores = util.pytorch_Dis(query_embedding, corpus_embeddings)
        cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(cos_scores, range(top_k))[0:top_k]
    elif (simtype == 'DPS'):
        cos_scores = util.pytorch_DPS(query_embedding, corpus_embeddings)
        cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    elif (simtype == 'BS'):
        cos_scores = util.pytorch_BS(query_embedding, corpus_embeddings)
        cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    elif (simtype == 'MD'):
        cos_scores = util.pytorch_Manhattan_Dis(query_embedding, corpus_embeddings)
        cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    elif (simtype == 'JS'):
        cos_scores = util.pytorch_Jaccard_Sim(query_embedding, corpus_embeddings)
        cos_scores = cos_scores.cpu()
        #We use np.argpartition, to only partially sort the top_k results
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    else:
        break

    print("\n\n\n\n")
    for qlen in range(0, len(queries_ori[i]), 50):
        print(show + 1 if qlen==0 else '', "Query:" if qlen==0 else '\t', queries_ori[i][qlen : qlen + 50])
    #print("\nTop 5 most similar sentences in corpus:")
    print('')
    no = 1
    for idx in top_results[0:top_k]:
        if (corpus[idx] == specialtest):
            continue
        if (queries_ori[i] == corpusList[idx].Situation):
            continue
        if (isinstance(corpusList[idx].Situation, float)):
            continue
        if (corpusList[idx].Situation.strip() == 'nan'):
            continue

        print(no, "  "
              , corpusList[idx].no,"\t"
              ,  corpusList[idx].reason
              , "\t(Score: %.4f)" % (cos_scores[idx]))
        for qlen in range(0, len(corpusList[idx].Situation.strip()), 50):
            print(corpusList[idx].Situation.strip()[qlen : qlen + 50])
        #print (corpusList[idx].Situation.strip())
        #    #if (len(corpus[idx].strip()[0:100]) > 100):
        #    print (corpus[idx].strip()[0:100])
        #else:
        #    print (corpus[idx].strip()[0:100])
        #if (tmpSpecial == ''):
        #    tmpSpecial = corpus[idx].strip()
        #    print(tmpSpecial)
        print('\n')
        no = no + 1
        if (no > 5):
            break

    show += 1
    tEnd2 = time.time()#計時結束
    print("COS相似計算時間", "%f sec" % (tEnd2 - tStart2))

    if (show >= max):break

In [ ]:

testtype = 2
simtype = 'MD'


In [52]:
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import KMeans
embedder = SentenceTransformer(berttype)
#embedder = SentenceTransformer('LSCBS')

Exception when trying to download https://sbert.net/models/bert-base-chinese.zip. Response 404


In [53]:
if (testtype != 0):
    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
else:
    corpus_embeddings = embedder.encode(corpus)

In [54]:
if (testtype == 1):

    tmparray = []
    for embedding in corpus_embeddings:
        tmparray.extend([[embedding[0], embedding[0]]])
    corpus_embeddings = np.array(tmparray)
elif(testtype == 2):
    queries = []
    queries_ori = []
    specialtest  ='。'
    queries_ori.append(specialtest)
    queries.append(specialtest)
    query_embedding = embedder.encode(queries[0], convert_to_tensor=True)
    if (simtype == 'COS'):
        cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
        cos_scores = cos_scores.cpu()
    elif (simtype == 'DIS'):
        cos_scores = util.pytorch_Dis(query_embedding, corpus_embeddings)
        cos_scores = cos_scores.cpu()
    elif (simtype == 'DPS'):
        cos_scores = util.pytorch_DPS(query_embedding, corpus_embeddings)
        cos_scores = cos_scores.cpu()
    elif (simtype == 'MD'):
        cos_scores = util.pytorch_Manhattan_Dis(query_embedding, corpus_embeddings)
        cos_scores = cos_scores.cpu()
    tmparray = []
    i = 0
    for a in corpus_embeddings:
        a = a.tolist()
        c = cos_scores[i].tolist()

        tmp = []
        for _a in a:
            tmp.append(_a)
        tmp.append(c)
        tmparray.extend([tmp])
        i+=1
    corpus_embeddings = np.array(tmparray)
      #tmparray.extend([[embedding, embedding[0]]])

In [55]:
from scipy import stats
import pandas as pd

def Entropy(labels):
    # 计算概率分布
    probs = pd.Series(labels).value_counts() / len(labels)
    # 计算底数为base的熵
    en = stats.entropy(probs)
    return en

class Cls(object):
    def __init__(self):
        self.reason = ""
        self.num = 0

In [56]:
specialcls = -1

num_clusters = 20
for ixxx in range(15):
    clustering_model = KMeans(n_clusters=num_clusters)
    clustering_model.fit(corpus_embeddings)
    cluster_assignment = clustering_model.labels_


    #%%

    clustered_sentences = [[] for i in range(num_clusters)]
    for sentence_id, cluster_id in enumerate(cluster_assignment):
        clustered_sentences[cluster_id].append(corpus[sentence_id])
        if (tmpSpecial == corpus[sentence_id]):
            specialcls = cluster_id

    no = 1
    clusterList = {}
    show = 0
    for sentence_id, cluster_id in enumerate(cluster_assignment):
        if cluster_id in clusterList:
            if corpusList[sentence_id].reason in clusterList[cluster_id]:
                clusterList[cluster_id][corpusList[sentence_id].reason] += 1
            else:
                clusterList[cluster_id][corpusList[sentence_id].reason] = 1
        else:
            clslist = {}
            clslist[corpusList[sentence_id].reason] = 1
            clusterList[cluster_id] = clslist

    idcnt = 0

    for key, value in sorted(clusterList.items()):
        clutotal = sum(value.values())
        for _key, _value in sorted(value.items(), key=lambda x: -x[1]):
            idcnt += _value
            clusterList[key][_key] = value[_key] / clutotal


    score = 0
    clucnt = 0
    clusterEntropy = {}
    for key, value in sorted(clusterList.items()):
        clusterEntropy[key] = Entropy(value)
        score += clusterEntropy[key]
        clucnt += 1

    print("Avg Entropy:\t", score/clucnt, "\n")




Avg Entropy:	 1.577436794790112 

Avg Entropy:	 1.5184302774164113 

Avg Entropy:	 1.526319171575302 

Avg Entropy:	 1.5696710218330603 

Avg Entropy:	 1.6114457027828082 

Avg Entropy:	 1.5322231120985292 

Avg Entropy:	 1.541670935610681 

Avg Entropy:	 1.5874168273421883 

Avg Entropy:	 1.5283090804345498 

Avg Entropy:	 1.5385401474404203 

Avg Entropy:	 1.5610628673604658 

Avg Entropy:	 1.5974062911958844 

Avg Entropy:	 1.5237749176537703 

Avg Entropy:	 1.4618511588827805 

Avg Entropy:	 1.4976027072527776 



In [25]:
specialcls = -1

num_clusters = 20
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_


In [26]:
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])
    if (tmpSpecial == corpus[sentence_id]):
        specialcls = cluster_id
        print (cluster_id)
for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster[0].strip()[0:100])
    print("")

Cluster  1
王炎權於民國100年4月8日晚間9時30分許起至翌日凌晨3時許止，在彰化縣芬園鄉路○段291巷15號之住處內飲酒後，已達不能安全駕駛動力交通工具之程度，竟仍駕駛車牌號碼為號自小客車上路，欲往南投縣日月

Cluster  2
丁係年滿十八歲之未成年人，明知潘志軒、真實姓名年籍不詳綽號「鴻哥」之成年男子及渠等所屬之詐欺集團，係利用電話假藉司法人員、警調機關偵辦案件名義，向不特定民眾詐欺取財，竟參與該詐欺集團，擔任取款之車手，

Cluster  3
被告丁為菲律賓籍看護工即告訴人甲雇主丙之胞兄，而告訴人係受僱在新北市區路0段00巷00號照顧被告之母乙。被告於民國101年11月9日17時許，在本件住處內因與告訴人發生爭執，且雙方語言不通，其明知人體

Cluster  4
林國銘可得而知FOXY檔案傳輸軟體，係一種以點對點技術下載或傳輸檔案之程式，當使用者執行FOXY軟體下載單一檔案時，於下載完成前、後，分別會儲存於該軟體預設之暫存資料夾及下載資料夾，且此2資料夾均經該

Cluster  5
邱建豪於民國106年5月1日下午5時15分許，在桃園市區路000號桃園市政府警察局桃園分局埔子派出所內之值班臺，明知員警徐芳振係依法執行職務之公務員，因不滿員警告以所詢事項應向其他單位辦理，竟基於妨害

Cluster  6
被告張芳碩明知改造槍枝為槍砲彈藥刀械管制條例所列管之違禁物，未經中央主管機關之許可，不得加以販賣，竟基於持有販賣改造槍枝之犯意，於民國104年3月中旬之某日，在其所經營位於臺中市區路0段000號1樓之

Cluster  7
黎桓銘承租並居住在臺中縣太平市自民國98年9月某日起至99年3月9日前之某日止之日間，先後3次，持其所有、租書使用之錦城漫畫影音會員卡1張，開啟陳廣致上開租屋處大門，自大門侵入該房屋，分別竊取陳廣致所

Cluster  8
康昭仁於「世旻設計有限公司」擔任業務經理一職，於民國99年8月間某日起，於網路YES123求職網站上張貼招募人才之訊息。適有代號0女子，於同一網站上張貼求職之訊息，康昭仁遂撥打電話予A女，約定於99年

Cluster  9
林秉鈞明知如附表一所示之商標，係該附表所示之商標權人向我國經濟部智慧財產局申請註冊並取得商標專用權之圖樣，且現仍在商標專用期限內，任何人未經商標權人之同意或授權，不得於同

In [27]:
class Cls(object):
    def __init__(self):
        self.reason = ""
        self.num = 0

In [28]:
no = 1
clusterList = {}
show = 0
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if (show < 5):
        print(no, "  "
                  , corpusList[sentence_id].no,"\t"
                  ,  corpusList[sentence_id].reason, "\n")
        if (len(corpusList[sentence_id].Situation.strip()) > 100):
            print(corpusList[sentence_id].Situation.strip()[0:100])
        else:
            print(corpusList[sentence_id].Situation.strip())
        no += 1
        print("")

        show +=1
    if cluster_id in clusterList:
        if corpusList[sentence_id].reason in clusterList[cluster_id]:
            clusterList[cluster_id][corpusList[sentence_id].reason] += 1
        else:
            clusterList[cluster_id][corpusList[sentence_id].reason] = 1
    else:
        clslist = {}
        clslist[corpusList[sentence_id].reason] = 1
        clusterList[cluster_id] = clslist

1    106,易,1093 	 妨害公務 

邱建豪於民國106年5月1日下午5時15分許，在桃園市區路000號桃園市政府警察局桃園分局埔子派出所內之值班臺，明知員警徐芳振係依法執行職務之公務員，因不滿員警告以所詢事項應向其他單位辦理，竟基於妨害

2    107,侵簡,3 	 妨害性自主 

余暐勝於民國107年3月間透過網路交友軟體「BEETALK」結識代號000之14歲以上未滿16歲少女，進而交往。詎余暐勝明知A女與其胞妹年紀相同，為就讀國中、未滿16歲之少女，竟仍基於對14歲以上未滿

3    100,中交簡,1002 	 公共危險 

王炎權於民國100年4月8日晚間9時30分許起至翌日凌晨3時許止，在彰化縣芬園鄉路○段291巷15號之住處內飲酒後，已達不能安全駕駛動力交通工具之程度，竟仍駕駛車牌號碼為號自小客車上路，欲往南投縣日月

4    100,中交簡,100 	 公共危險 

林耀明於民國99年10月22日下午4時許起至5時許止，在臺中市區路與豐樂路交岔路口附近之某工地內，與友人飲用酒類後，已達不能安全駕駛動力交通工具之程度，猶於同日下午6時許，駕駛車牌號碼號自用小客車上路

5    100,中交簡,1016 	 公共危險 

朱建鴻於民國100年4月19日晚間9時許起，在臺中市大雅區清泉崗友人住處內飲用啤酒，至同日晚間10時許已達不能安全駕駛動力交通工具之程度，猶於飲畢後騎乘車牌號碼號重型機車上路欲返回住處，嗣於同日晚間1



In [29]:
idcnt = 0

for key, value in sorted(clusterList.items()):
    print(key, " ")
    clutotal = sum(value.values())
    print(clutotal)
    for _key, _value in sorted(value.items(), key=lambda x: -x[1]):
        idcnt += _value
        clusterList[key][_key] = value[_key] / clutotal
        print(_key, " ", _value)
        #print(_key, " ", _value, "\nP= ",clusterList[key][_key])
    print(" ")

0  
264
公共危險   250
過失傷害   6
偽造文書   5
妨害性自主   1
妨害公務   1
貪污   1
 
1  
126
詐欺   97
妨害風化   9
恐嚇取財   4
賭博   3
竊盜   3
殺人未遂   3
恐嚇   2
妨害自由   2
妨害名譽   1
強盜   1
公共危險   1
 
2  
411
傷害   158
殺人未遂   76
毀損   51
妨害名譽   50
妨害自由   31
毀棄損壞   10
殺人   9
妨害公務   9
妨害性自主   8
恐嚇   4
強盜   2
搶奪   1
竊盜   1
詐欺   1
 
3  
194
妨害名譽   130
著作權法   26
妨害自由   15
妨害風化   4
詐欺   3
恐嚇取財   3
妨害性自主   2
殺人   2
毒品危害   2
毀損   2
殺人未遂   1
偽造文書   1
強盜   1
公共危險   1
恐嚇   1
 
4  
306
妨害公務   253
偽造文書   12
妨害自由   12
妨害名譽   7
傷害   6
公共危險   4
毀損   3
殺人未遂   2
恐嚇   2
過失傷害   1
竊盜   1
侵占   1
妨害性自主   1
毒品危害   1
 
5  
219
槍砲彈藥刀械   213
強盜   3
殺人未遂   2
貪污   1
 
6  
392
偽造文書   106
侵占   92
詐欺   50
妨害自由   36
毀損   29
竊盜   16
妨害名譽   14
妨害公務   9
著作權法   5
槍砲彈藥刀械   5
妨害風化   5
賭博   4
毀棄損壞   3
貪污   3
強盜   2
恐嚇   2
恐嚇取財   2
商標法   2
醫師法   2
殺人未遂   1
傷害   1
搶奪   1
毒品危害   1
公共危險   1
 
7  
300
侵占   85
偽造文書   69
著作權法   30
妨害名譽   28
妨害自由   23
詐欺   14
毀損   12
妨害風化   8
商標法   6
妨害公務   4
妨害性自主   3
毀棄損壞   3
過失致死   3
貪污   2
竊盜   2
傷害   2
賭博   2
公共危險   1
恐嚇取財   1
醫師法 

In [30]:
from scipy import stats
import pandas as pd

def Entropy(labels):
    # 计算概率分布
    probs = pd.Series(labels).value_counts() / len(labels)
    # 计算底数为base的熵
    en = stats.entropy(probs)
    return en

In [31]:
score = 0
clucnt = 0
clusterEntropy = {}
for key, value in sorted(clusterList.items()):
    clusterEntropy[key] = Entropy(value)
    score += clusterEntropy[key]
    clucnt += 1

print("Avg Entropy:\t", score/clucnt, "\n")

for key, value in sorted(clusterEntropy.items()):
    print("Clu", key, "Entropy:\t", value)

Avg Entropy:	 1.585726918841268 

Clu 0 Entropy:	 1.2424533248940002
Clu 1 Entropy:	 1.6726254461503207
Clu 2 Entropy:	 2.3046193848206715
Clu 3 Entropy:	 1.7094727765842133
Clu 4 Entropy:	 1.8662160235859526
Clu 5 Entropy:	 1.3862943611198906
Clu 6 Entropy:	 2.312365899036895
Clu 7 Entropy:	 2.3594656874870674
Clu 8 Entropy:	 0.0
Clu 9 Entropy:	 0.6365141682948128
Clu 10 Entropy:	 2.0771015748862114
Clu 11 Entropy:	 0.8675632284814613
Clu 12 Entropy:	 2.857102837442002
Clu 13 Entropy:	 2.476220649598165
Clu 14 Entropy:	 0.6931471805599453
Clu 15 Entropy:	 2.0253262207700677
Clu 16 Entropy:	 0.6365141682948128
Clu 17 Entropy:	 2.36173836677077
Clu 18 Entropy:	 1.6674619334292948
Clu 19 Entropy:	 0.5623351446188083
